In [23]:
import numpy as np
import pandas as pd

train_set_days = pd.read_csv("train_set_days.csv")
train_set_weeks = pd.read_csv("train_set_weeks.csv")
train_set_sfa = pd.read_csv("train_set_sfa.csv")
info_business_points = pd.read_csv("info_business_points.csv")
info_groups = pd.read_csv("info_groups.csv")
info_items = pd.read_csv("info_items.csv")

test_set_weeks = pd.read_csv('test_set_weeks.csv')

In [ ]:
### .._set_weeks ###

In [8]:
def prepare_set_weeks(df_param):
    df_param.wk = df_param.wk.astype(str)
    df_param.wk = df_param.wk.str[:4] + ',' + df_param.wk.str[4:]
    df_param[['wk_year', 'year']] = df_param.wk.str.split(',', expand=True)
    df_param = df_param.drop('wk', axis=1)
    df_param.wk_year = df_param.wk_year.astype(int)
    df_param.year = df_param.year.astype(int)
    return df_param

In [9]:
train_set_weeks = prepare_set_weeks(train_set_weeks)
train_set_weeks.head(5)

,id,N wk,idFilial,KanalDB,idSubGrp,value,wk_year,year
0,0,1,9,2,3,3560.0,2014,1
1,1,2,9,2,3,7120.0,2014,2
2,2,3,9,2,3,57672.0,2014,3
3,3,4,9,2,3,37380.0,2014,4
4,4,5,9,2,3,80990.0,2014,5


In [30]:
train = train_set_weeks.drop(['id','value'], axis=1)
target = train_set_weeks.value

In [27]:
test_set_weeks = prepare_set_weeks(test_set_weeks)
test_set_weeks.head(5)

,id,N wk,idFilial,KanalDB,idSubGrp,wk_year,year
0,0,184,9,2,3,2017,29
1,1,185,9,2,3,2017,30
2,2,186,9,2,3,2017,31
3,3,187,9,2,3,2017,32
4,4,188,9,2,3,2017,33


In [32]:
test = test_set_weeks.drop(['id'], axis=1)

In [11]:
### Train_set_sfa ###

In [24]:
def prepare_set_sfa(df_param):
    notnull_idxs = pd.notnull(df_param.calc_share)
    df_param.calc_share[notnull_idxs] = df_param.calc_share[notnull_idxs].astype(str)
    df_param.calc_share[notnull_idxs] = df_param.calc_share[notnull_idxs].str[:-1]
    df_param.calc_share = df_param.calc_share.astype(float)
    df_param.calc_share /= 100
    df_param.visit_date = pd.to_datetime(df_param.visit_date, format='%d.%m.%Y')
    df_param['day'] = df_param.visit_date.dt.day
    df_param['month'] = df_param.visit_date.dt.month
    df_param['year'] = df_param.visit_date.dt.year
    df_param = df_param.drop('visit_date', axis=1)
    return df_param

In [25]:
train_set_sfa = prepare_set_sfa(train_set_sfa)
train_set_sfa[pd.notnull(train_set_sfa.calc_share)].head(10)

/Users/irina/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/irina/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,id,idBP,idFilial,calc_share,visits_count,calls_count,day,month,year
48916,48916,66988,4,0.49,1,0,1,10,2016
48917,48917,70600,4,0.61,1,0,1,10,2016
48919,48919,5177,10,0.70,0,1,1,10,2016
48920,48920,5266,10,0.33,0,1,1,10,2016
48921,48921,6234,10,0.67,0,1,1,10,2016
48926,48926,801,4,0.51,1,0,3,10,2016
48927,48927,1302,4,0.52,1,0,3,10,2016
48928,48928,4624,4,0.19,1,0,3,10,2016
48930,48930,59796,4,0.57,1,0,3,10,2016
48931,48931,61166,4,1.00,1,0,3,10,2016


In [ ]:
### Spliting ###

In [ ]:
from sklearn.

In [ ]:
### CatBoostRegressor ###

In [33]:
from catboost import CatBoostRegressor
model = CatBoostRegressor()